# Initial Google colab setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# first time setup

# !pip install -U nltk
# !pip install appmode
# jupyter nbextension enable --py --sys-prefix appmode
# jupyter serverextension enable --py --sys-prefix appmode

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import os
from os import mkdir
import io
from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Helper Functions

In [ ]:
# Helper functions

#################### HELPERS: messages ####################

def print_warning(arg_warning_msg):
	print("\n##################################################")
	print("##### WARNING: " + arg_warning_msg)
	print("##################################################\n")

def print_error(arg_error_msg):
	print("\n##################################################")
	print("########## ERROR: " + arg_error_msg)
	print("##################################################\n")

#################### HELPERS: dictionary ####################

# this adds an keyed empty dictionary to a dictionary (of dictionaries)
def add_dict_to_ddict(arg_ddict, arg_dict_key):
	if arg_dict_key not in arg_ddict:
		arg_ddict[arg_dict_key] = {}

# this initialises a key in a dictionary with a provided value 
def init_dict_key(arg_dict, arg_key, arg_init_val):
	if arg_key not in arg_dict:
		arg_dict[arg_key] = arg_init_val

# this adds a value into a list within a dictionary 
def append_to_list_in_dict(arg_dict, arg_key, arg_val):
	if arg_key not in arg_dict:
		arg_dict[arg_key] = []
	arg_dict[arg_key].append(arg_val)

# this adds a value to a list of unique values
def append_uniq_to_list(arg_list, arg_val):
	if arg_val not in arg_list:
		arg_list.append(arg_val)
	if len(arg_list) != len(set(arg_list)):
		print_warning("list contains duplicates, expected list of unique items")

# this adds a value to a dictionary
def add_val_to_dict(arg_dict, arg_key, arg_val, arg_uniq = 0):
	try:
		if arg_key in arg_dict and arg_uniq == 0:
			arg_dict[arg_key] = arg_dict[arg_key] + arg_val
		if arg_key in arg_dict and arg_uniq == 1:
			print_warning("this key " + str(arg_key) + " is not unique in the sentence")
		if arg_key not in arg_dict:
			arg_dict[arg_key] = arg_val
	except:
		print_error("unknown error at function: add_val_to_dict")

#################### PRINT ####################

def print_dict(arg_dict, arg_dict_name = ""):
	print(arg_dict_name)
	print(arg_dict, "\n")

def print_dict_lines(arg_dict, arg_dict_name = ""):
	print(arg_dict_name)
	for tup in arg_dict.items():
		print("    \"" + str(tup[0]) + "\": " + str(tup[1]))
	print("\n")

def print_ddict(arg_ddict, arg_ddict_name = "", arg_str_limit = 10000):
	print(arg_ddict_name)
	for tup in arg_ddict.items():
		print(str(tup)[ : min(len(str(tup)), arg_str_limit)])
	
#################### TIME ####################

import time;
def fill_time(arg_time):
	if len(arg_time) < 2:
		res = "0" + arg_time
	else:
		res = arg_time
	return res

def absvalue(arg_time):
  if arg_time < 0:
    return (24 + arg_time)
  else:
    return arg_time

def get_timestamp(arg_tz):
  localtime = time.localtime(time.time())
  time_str = time.asctime(time.localtime(time.time()))
  y = fill_time(str(localtime[0])[2:4])
  mo = fill_time(str(localtime[1]))
  d = fill_time(str(localtime[2]))
  h = fill_time(str(absvalue(localtime[3] + arg_tz)))
  min = fill_time(str(localtime[4]))
  s = fill_time(str(localtime[5]))
  timestamp = y + mo + d + "_" + h + min + s
  # print("Local current time :", time_str, "timestamp:", timestamp)
  return timestamp

# Global Variables

In [ ]:
GLOBAL_pickle = True
global_pickle_file = "global"

GLOBAL_timestamp = get_timestamp(-5)
GLOBAL_trainfiles = "data_lite_2"
if GLOBAL_pickle:
  GLOBAL_picklefiles = "pickle_lite_2_1498"
  pickle_path = os.path.join(os.getcwd(), "drive", "My Drive/CS 4701/pickle/" + GLOBAL_picklefiles) # replace based on your Google drive organization
  trained_pickles_path = pickle_path
  # pickle_path = os.path.join(os.getcwd(), "drive", "My Drive/CS 4701/pickle") # replace based on your Google drive organization
  print("trained pickles:", len(os.listdir(pickle_path)), "pickles, path:", pickle_path, "\npickles:", os.listdir(pickle_path))

# all_genres = "All Genres"
synonyms_dict = {
  "\'\'": "\"",
  "``" : "\"",
  "“" : "\"",
  "\'\'" : "\"",
  "”" : "\"",
}
# print(synonyms_dict)
GLOBAL_sent_ends = ["</s>", ".", "?", "!"]


train_path = os.path.join(os.getcwd(), "drive", "My Drive/CS 4701/" + GLOBAL_trainfiles + "/train") # replace based on your Google drive organization
print("training dataset:", len(os.listdir(train_path)), "genres, path:", train_path, "\nfolders:", os.listdir(train_path))

LOCAL_path = GLOBAL_timestamp + "/"
GLOBAL_genres = []
GLOBAL_threshold = 0.5
GLOBAL_num_threshold = 1998

GLOBAL_UNKs = {}

GLOBAL_dataset_info = {}

GLOBAL_blacklist = ["--"]


trained pickles: 11 pickles, path: /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498 
pickles: ['global', 'adventure', 'christmas', 'biography', 'childrens_fiction', 'detective', 'fantasy', 'horror', 'poetry', 'philosophy', 'science_fiction']
training dataset: 10 genres, path: /content/drive/My Drive/CS 4701/data_lite_2/train 
folders: ['christmas', 'fantasy', 'horror', 'adventure', 'poetry', 'biography', 'detective', 'philosophy', 'childrens_fiction', 'science_fiction']


# Document and N-gram Extraction

In [ ]:
# document extracter functions
def init_genres(arg_dataset_path):
  genre_list = list(os.listdir(arg_dataset_path))
  genre_list = list(filter(lambda filename: filename != ".DS_store", genre_list))
  genre_list.sort()
  return genre_list

def init_books(arg_genre):
  genre_path = train_path + "/" + arg_genre
  genre_books_unfiltered = os.listdir(genre_path)
  genre_books = list(filter(lambda x: ".txt" in x, genre_books_unfiltered))
  genre_books.sort()
  return genre_path, genre_books

def trim_textdata(arg_raw_text, arg_genre, arg_bookfile):
  if "START OF THIS PROJECT GUTENBERG EBOOK" in arg_raw_text and len(arg_raw_text.split("START OF THIS PROJECT GUTENBERG EBOOK")) == 2 and len(arg_raw_text.split("END OF THIS PROJECT GUTENBERG EBOOK")) == 2:
    textdata_0 = arg_raw_text.split("START OF THIS PROJECT GUTENBERG EBOOK")[1]
    textdata = textdata_0.split("END OF THIS PROJECT GUTENBERG EBOOK")[0]
  elif len(arg_raw_text.split("PROJECT GUTENBERG EBOOK")) == 3:
    textdata = arg_raw_text.split("PROJECT GUTENBERG EBOOK")[1]
  else:
    print_error("Text Data Parsing Error at " + arg_genre + " : " + str(arg_bookfile))
    return None
  if len(textdata) == 0:
    print_error("Text Data Parsing Error at " + arg_genre + " : " +  str(arg_bookfile))
  return textdata


In [ ]:
# check if the books fulfill layout
# append genre to list of genres  
def check_books(arg_dataset_path, arg_check_books = False):
  for genre in init_genres(arg_dataset_path):
    GLOBAL_genres.append(genre)
    if arg_check_books:
      genre_path, bookfiles = init_books(genre)
      for bookfile in bookfiles:
        with io.open(os.path.join(genre_path, bookfile), encoding='utf8') as f:
          textdata = trim_textdata(f.read(), genre, bookfile)

check_books(train_path)

In [ ]:

def get_synonyms(arg_tok, arg_synonyms_dict):
  if arg_tok in arg_synonyms_dict:
    # print(arg_tok, arg_synonyms_dict[arg_tok])
    return arg_synonyms_dict[arg_tok]
  else:
    return arg_tok

# gets aggregate as well as genre-specific ngram data for frequency (how many of this unigram) and count (how many of any unigrams are in a genre)
def get_ngrams(arg_sent, arg_genre, ugf, bgf, tgf, ugc, bgc, tgc):
  for i in range(0, len(arg_sent)):
    unigram = get_synonyms(arg_sent[i], synonyms_dict)
    if unigram == "<total>":
      print_warning("<total> tag in text")
    # add_val_to_dict(ugc, all_genres, 1)
    # add_val_to_dict(ugf[all_genres], unigram, 1)
    add_val_to_dict(ugc, arg_genre, 1)
    add_val_to_dict(ugf[arg_genre], unigram, 1)
    if i > 0:
      bigram = (get_synonyms(arg_sent[i - 1], synonyms_dict), unigram)
      # add_val_to_dict(bgc, all_genres, 1)
      # add_val_to_dict(bgf[all_genres], bigram, 1)
      add_val_to_dict(bgc, arg_genre, 1)
      add_val_to_dict(bgf[arg_genre], bigram, 1) 
    if i > 1:
      trigram = (get_synonyms(arg_sent[i - 2], synonyms_dict), get_synonyms(arg_sent[i - 1], synonyms_dict), unigram)
      # add_val_to_dict(tgc, all_genres, 1)
      # add_val_to_dict(tgf[all_genres], trigram, 1)
      add_val_to_dict(tgc, arg_genre, 1)
      add_val_to_dict(tgf[arg_genre], trigram, 1)

"""sents dict { 
  genre 1 book sentence tokens:
    list of lists containing all sentences of books in genre 1[
      list of sentence tokens [book1_sent_1 toks], [book1_sent_2 toks], ... , [book1_sent_last toks], [book2_sent_1 toks], ... , [book_last_sent_1 toks], ... , [book_last_sent_last_toks]
    ] ,
  genre 2 book sentence tokens:
    list of lists containing all sentences of books in genre 2[
      list of sentence tokens [book1_sent_1 toks], [book1_sent_2 toks], ... , [book1_sent_last toks], [book2_sent_1 toks], ... , [book_last_sent_1 toks], ... , [book_last_sent_last_toks]
    ] ,
  ...
  genre last book sentence tokens:
    list of lists containing all sentences of books in genre G (for G genres)[
      list of sentence tokens [book1_sent_1 toks], [book1_sent_2 toks], ... , [book1_sent_last toks], [book2_sent_1 toks], ... , [book_last_sent_1 toks], ... , [book_last_sent_last_toks]
    ]
}"""

# def check_books(arg_dataset_path):
#   for genre in init_genres(arg_dataset_path):
#     genre_path, bookfiles = init_books(genre)
#     for bookfile in bookfiles:
#       with io.open(os.path.join(genre_path, bookfile), encoding='utf8') as f:
#         textdata = trim_textdata(f.read())

def get_most_freq_toks(arg_dataset_path):
  tf = {}
  for genre in init_genres(arg_dataset_path):
    print("Handling unknowns, genre:", genre)
    # add genre dicts to ddicts 
    # GLOBAL_genres.append(genre)
    add_dict_to_ddict(tf, genre)
    add_dict_to_ddict(GLOBAL_UNKs, genre)
    add_dict_to_ddict(GLOBAL_dataset_info, genre)
    genre_path, bookfiles = init_books(genre)
    for bookfile in bookfiles:
      with io.open(os.path.join(genre_path, bookfile), encoding='utf8') as f:
        textdata = trim_textdata(f.read(), genre, bookfile)
        for sent in sent_tokenize(textdata):
          sent_toks = list(map(str.lower, word_tokenize(sent)))
          for tok in sent_toks:
            add_val_to_dict(tf[genre], tok, 1)
  most_freq_toks = {}
  for (genre, tfdict) in tf.items():
    add_dict_to_ddict(most_freq_toks, genre)
    # {tok0 : freq0 for tok0, freq0 in sorted(tfdict.items(), key = lambda item: item[1], reverse = True)}
    tflist0 = list({tok0 : freq0 for tok0, freq0 in sorted(tfdict.items(), key = lambda item: item[1], reverse = True)}.items())
    threshold = min(GLOBAL_num_threshold, int(GLOBAL_threshold * len(tflist0) + 1))
    tflist = tflist0[0 : threshold]
    for (tok, freq) in tflist:
      most_freq_toks[genre][tok] = freq
  print(most_freq_toks)
  return most_freq_toks 

def extract_sent_tokens(ugf, bgf, tgf, ugc, bgc, tgc, arg_dataset_path, arg_tfdict):
  print("extracting from:", train_path)
  for genre in init_genres(arg_dataset_path):
    # init_dict_key(arg_sents_dict, genre, [])
    add_dict_to_ddict(ugf, genre)
    add_dict_to_ddict(bgf, genre)
    add_dict_to_ddict(tgf, genre)
    init_dict_key(ugc, genre, 0)
    init_dict_key(bgc, genre, 0)
    init_dict_key(tgc, genre, 0)
    genre_path, bookfiles = init_books(genre)
    print("\ngenre, genre path:", genre, genre_path, os.listdir(genre_path))
    for book_idx in range(0, len(bookfiles)):
      bookfile = bookfiles[book_idx] 
      add_val_to_dict(GLOBAL_dataset_info[genre], "book_count", 1)
      with io.open(os.path.join(genre_path, bookfile), encoding='utf8') as f:
        textdata = trim_textdata(f.read(), genre, bookfile)
        TEMP_len = 0
        # TEMP_trimmed = 0
        TEMP_wordcount = 0
        TEMP_unkcount = 0
        TEMP_len += len(textdata)
        if len(textdata) == 0:
          print_error("Text Data Parsing Error")
        for sent in sent_tokenize(textdata):
          sent_toks0 = list(map(str.lower, word_tokenize(sent)))
          sent_toks = []
          for tok0 in sent_toks0:
            if tok0 in arg_tfdict[genre]:
              sent_toks.append(tok0)
            else:
              if tok0 not in GLOBAL_blacklist:
                sent_toks.append("<UNK>")
                add_val_to_dict(GLOBAL_UNKs[genre], tok0, 1)
                TEMP_unkcount += 1
          sent_toks.insert(0, "<s>")
          sent_toks.insert(0, "<s>")
          sent_toks.append("</s>")
          # arg_sents_dict[genre].append(sent_toks)
          get_ngrams(sent_toks, genre, ugf, bgf, tgf, ugc, bgc, tgc)
          TEMP_wordcount += len(sent_toks)
      add_val_to_dict(GLOBAL_dataset_info[genre], "token_count", TEMP_wordcount)
      add_val_to_dict(GLOBAL_dataset_info[genre], "unk_count", TEMP_unkcount)
      print("  extracted book", str(book_idx + 1), "out of", str(len(bookfiles)), ":", str(bookfile), \
            "; length:", TEMP_wordcount, "tokens (", TEMP_unkcount, "unknowns ),", TEMP_len, "characters ...")
    GLOBAL_dataset_info[genre]["booklength"] = int(GLOBAL_dataset_info[genre]["token_count"] / GLOBAL_dataset_info[genre]["book_count"])


# Pickling Jar

In [ ]:
from typing import Callable

def output_dict_textfile(arg_dict, arg_fname):
  with open(arg_fname, "w") as file:
    file.write("dictionary of genres, list (sents), of list (sent toks)")
    for (genre, genre_sents) in arg_dict.items():
      file.write("\n\ngenre: " + genre + "(" + str(len(genre_sents)) + " sents)")
      for sent_idx in range(len(genre_sents)):
        if sent_idx < 100 or (len(genre_sents) - sent_idx) < 100:
          file.write("\n  " + str(sent_idx) + " : " + str(genre_sents[sent_idx]))

# output_dict_textfile(train_sent_toks_by_genre, "cs4701_toydict_" + get_timestamp(-4))

In [ ]:
def output_tok_dict_textfile(arg_ugf_dict, arg_bgf_dict, arg_tgf_dict, arg_fname):
  with open(arg_fname, "w") as file:
    file.write("dictionary of genres, ngram frequencies")
    for (genre, tok_freqs) in arg_ugf_dict.items():
      file.write("\n\ngenre: " + genre + "(" + str(len(list(tok_freqs.items()))) + " unigram types)")
      for (tok, freq) in tok_freqs.items():
        if freq >= 10:
          file.write("\n  " + str(tok) + " : " + str(freq))
    for (genre, tok_freqs) in arg_bgf_dict.items():
      file.write("\n\ngenre: " + genre + "(" + str(len(list(tok_freqs.items()))) + " bigram types)")
      for (tok, freq) in tok_freqs.items():
        if freq >= 10:
          file.write("\n  " + str(tok) + " : " + str(freq))
    for (genre, tok_freqs) in arg_tgf_dict.items():
      file.write("\n\ngenre: " + genre + "(" + str(len(list(tok_freqs.items()))) + " trigram types)")
      for (tok, freq) in tok_freqs.items():
        if freq >= 10:
          file.write("\n  " + str(tok) + " : " + str(freq))
        

In [ ]:
# {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}

def sort_tok_dicts(arg_ugf_dict, arg_bgf_dict, arg_tgf_dict):
  ugt_dict = {}
  bgt_dict = {}
  tgt_dict = {}
  ugf_dict = {}
  bgf_dict = {}
  tgf_dict = {}
  for (genre, tok_freqs) in arg_ugf_dict.items():
    ugt_dict[genre] = len(list(tok_freqs.items()))
    ugf_dict[genre] = {ngram : freq for ngram, freq in sorted(tok_freqs.items(), key = lambda item: item[1], reverse = True)}
  for (genre, tok_freqs) in arg_bgf_dict.items():
    bgt_dict[genre] = len(list(tok_freqs.items()))
    bgf_dict[genre] = {ngram : freq for ngram, freq in sorted(tok_freqs.items(), key = lambda item: item[1], reverse = True)}
  for (genre, tok_freqs) in arg_tgf_dict.items():
    tgt_dict[genre] = len(list(tok_freqs.items()))
    tgf_dict[genre] = {ngram : freq for ngram, freq in sorted(tok_freqs.items(), key = lambda item: item[1], reverse = True)}
  print("", ugt_dict, "\n", bgt_dict, "\n", tgt_dict)
  return ugf_dict, bgf_dict, tgf_dict, ugt_dict, bgt_dict, tgt_dict

# ugf_dict, bgf_dict, tgf_dict, ugt_dict, bgt_dict, tgt_dict = sort_tok_dicts(ugf_dict, bgf_dict, tgf_dict)

# output_tok_dict_textfile(ugf_dict, bgf_dict, tgf_dict, "cs4701_toytoks_" + get_timestamp(-5))
# output_tok_dict_textfile(ugf_dict, bgf_dict, tgf_dict, "cs4701_toytoks_" + get_timestamp(8)) # beijing time

## New method

In [ ]:
def write_pickle(arg_genre, arg_type, arg_ngram, arg_freq):
  ngram = ""
  if type(arg_ngram) == str:
    ngram = arg_ngram
  else:
    for tok in arg_ngram:
      ngram += tok + " "
  res = arg_genre + " " + arg_type + " " + str(arg_freq) + " " + ngram 
  res = res.strip()
  return res

def make_pickles(arg_ugf_dict, arg_bgf_dict, arg_tgf_dict, arg_pickle_path):
  for genre in GLOBAL_genres:
    with open(LOCAL_path + genre, "w") as file:
      for (ug, freq) in arg_ugf_dict[genre].items():
        res = write_pickle(genre, "ug", ug, freq)
        file.write(res)
        file.write("\n")
      for (bg, freq) in arg_bgf_dict[genre].items():
        res = write_pickle(genre, "bg", bg, freq)
        file.write(res)
        file.write("\n")
      for (tg, freq) in arg_tgf_dict[genre].items():
        res = write_pickle(genre, "tg", tg, freq)
        file.write(res)
        file.write("\n")

def write_line(arg_genre, arg_data_type, arg_name, arg_key, arg_value):
  res = "" 
  res += arg_genre + " " + arg_data_type + " " + arg_name + " " + str(arg_key) + " " + str(arg_value)
  return res

def write_global(arg_fname):
  with open(LOCAL_path + arg_fname, "w") as file:
    for genre, tf in most_freq.items():
      for tok, freq in tf.items():
        res = write_line(genre, "dict", "most_freq", tok, freq)
        file.write(res)
        file.write("\n")
    for genre, unkf in GLOBAL_UNKs.items():
      for tok, freq in unkf.items():
        res = write_line(genre, "list", "global_unks", tok, freq)
        file.write(res)
        file.write("\n")

def read_pickle(arg_line):
  line = arg_line.split(" ")
  genre = line[0]
  ngram_type = line[1]
  freq = int(line[2])
  if line[1] == "ug":
    ngram = line[3]
  elif line[1] == "bg":
    ngram = (line[3], line[4])
  elif line[1] == "tg":
    ngram = (line[3], line[4], line[5])
  return genre, ngram_type, freq, ngram

def get_pickles(arg_pickle_path):
  pickles_unfiltered = os.listdir(arg_pickle_path)
  pickles = list(filter(lambda x: "global" not in x and ".DS" not in x, pickles_unfiltered))
  pickles.sort()
  ugt = {genre : {} for genre in GLOBAL_genres}
  bgt = {genre : {} for genre in GLOBAL_genres}
  tgt = {genre : {} for genre in GLOBAL_genres}
  ugf = {genre : {} for genre in GLOBAL_genres}
  bgf = {genre : {} for genre in GLOBAL_genres}
  tgf = {genre : {} for genre in GLOBAL_genres}
  for pickle in pickles:
    with io.open(os.path.join(pickle_path, pickle), encoding='utf8') as f:
      print(pickle, pickle_path)
      lines = f.readlines()
      for i in range(len(lines) - 1): # exclude last line since it is blank
        genre, ngram_type, freq, ngram = read_pickle(lines[i].strip())
        if ngram_type == "ug":
          ugf[genre][ngram] = freq
        elif ngram_type == "bg":
          bgf[genre][ngram] = freq
        elif ngram_type == "tg":
          tgf[genre][ngram] = freq
  for (genre, tok_freqs) in ugf.items():
    ugt[genre] = len(list(tok_freqs.items()))
    ugf[genre] = {ngram : freq for ngram, freq in sorted(tok_freqs.items(), key = lambda item: item[1], reverse = True)}
  for (genre, tok_freqs) in bgf.items():
    bgt[genre] = len(list(tok_freqs.items()))
    bgf[genre] = {ngram : freq for ngram, freq in sorted(tok_freqs.items(), key = lambda item: item[1], reverse = True)}
  for (genre, tok_freqs) in tgf.items():
    tgt[genre] = len(list(tok_freqs.items()))
    tgf[genre] = {ngram : freq for ngram, freq in sorted(tok_freqs.items(), key = lambda item: item[1], reverse = True)}
  print("", ugt, "\n", bgt, "\n", tgt)
  return ugf, bgf, tgf, ugt, bgt, tgt

def read_global(arg_line):
  line = arg_line.split(" ")
  # print(line)
  genre = line[0]
  data_type = line[1]
  name = line[2]
  tok = line[3]
  val = line[4]
  return genre, data_type, name, tok, val

def get_globals(arg_fname):
  most_freq = {}
  global_unks = {}
  for genre in GLOBAL_genres:
    add_dict_to_ddict(most_freq, genre)
    add_dict_to_ddict(global_unks, genre)
  with io.open(os.path.join(pickle_path, arg_fname), encoding='utf8') as f:
    lines = f.readlines()
    for i in range(len(lines) - 1): # exclude last line since it is blank
      genre, data_type, name, tok, val = read_global(lines[i].strip())
      if name == "most_freq":
        most_freq[genre][tok] = int(val)
      elif name == "global_unks":
        global_unks[genre][tok] = int(val)
  return most_freq, global_unks



In [ ]:
from google.colab import files
import shutil

if GLOBAL_pickle:
  most_freq, GLOBAL_UNKs = get_globals(global_pickle_file)
  ugf_dict, bgf_dict, tgf_dict, ugt_dict, bgt_dict, tgt_dict = get_pickles(trained_pickles_path)
else:
  mkdir(GLOBAL_timestamp)
  most_freq = get_most_freq_toks(train_path)
  # how many ug, bg, tg, for specific ug, bg, tg 
  ugf_dict = {}
  bgf_dict = {}
  tgf_dict = {}

  # how many ug, bg, tg, for all ug, bg, tg 
  ugc_dict = {}
  bgc_dict = {}
  tgc_dict = {}
  extract_sent_tokens(ugf_dict, bgf_dict, tgf_dict, ugc_dict, bgc_dict, tgc_dict, train_path, most_freq)
  ugf_dict, bgf_dict, tgf_dict, ugt_dict, bgt_dict, tgt_dict = sort_tok_dicts(ugf_dict, bgf_dict, tgf_dict)
  print_ddict(GLOBAL_dataset_info)
  write_global("global")
  make_pickles(ugf_dict, bgf_dict, tgf_dict, LOCAL_path)
  shutil.make_archive(LOCAL_path, 'zip', LOCAL_path)

for genre, tf_dict in most_freq.items():
  print(genre, len(list(tf_dict.items())), tf_dict)
  # for tok, freq in tf_dict.items():
  #   if tok not in ugf_dict[genre]:
  #     print(genre, tok, freq)


adventure /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
biography /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
childrens_fiction /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
christmas /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
detective /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
fantasy /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
horror /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
philosophy /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
poetry /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
science_fiction /content/drive/My Drive/CS 4701/pickle/pickle_lite_2_1498
 {'adventure': 1498, 'biography': 1498, 'childrens_fiction': 1500, 'christmas': 1498, 'detective': 1500, 'fantasy': 1498, 'horror': 1498, 'philosophy': 1498, 'poetry': 1498, 'science_fiction': 1498} 
 {'adventure': 145400, 'biography': 120501, 'childrens_fiction': 96959, 'christmas': 65117, 'detective': 93223, '

In [ ]:
# index and reverse index 

import numpy as np

# functions to construct index dicts and rev. index dicts
def index_dict(arg_ngf_smk_dict):
  idx_dict = {}
  inv_idx_dict = {}
  for (genre, tok_freq_dict) in arg_ngf_smk_dict.items():
    idx_dict[genre] = {}
    inv_idx_dict[genre] = {}
    kv_list = list(tok_freq_dict.items())
    for i in range(len(kv_list)):
      if i not in idx_dict[genre]:
        idx_dict[genre][i] = kv_list[i][0]
      else:
        print("########## ERROR: states contains dupes ##########")
    for i in range(len(kv_list)):
      if kv_list[i][0] not in inv_idx_dict[genre]:
        inv_idx_dict[genre][kv_list[i][0]] = i
      else:
        print("########## ERROR: states contains dupes ##########")
  return idx_dict, inv_idx_dict

ug_idxd, ug_ridxd = index_dict(ugf_dict)
bg_idxd, bg_ridxd = index_dict(bgf_dict)
tg_idxd, tg_ridxd = index_dict(tgf_dict)

print(str(ug_idxd)[:10000])
print(str(ug_ridxd)[:10000])
print(str(bg_idxd)[:10000])
print(str(bg_ridxd)[:10000])
print(str(tg_idxd)[:10000])
print(str(tg_ridxd)[:10000])

# for (genre, tok_idx) in ug_idxd.items():
#   print(genre, len(tok_idx.items()))
# for (genre, tok_idx) in ug_ridxd.items():
#   print(genre, len(tok_idx.items()))
# for (genre, tok_idx) in bg_idxd.items():
#   print(genre, len(tok_idx.items()))
# for (genre, tok_idx) in bg_ridxd.items():
#   print(genre, len(tok_idx.items()))
# for (genre, tok_idx) in tg_idxd.items():
#   print(genre, len(tok_idx.items()))
# for (genre, tok_idx) in tg_ridxd.items():
#   print(genre, len(tok_idx.items()))

{'adventure': {0: '<UNK>', 1: '<s>', 2: ',', 3: 'the', 4: '</s>', 5: '.', 6: '"', 7: 'of', 8: 'and', 9: 'to', 10: 'a', 11: 'i', 12: 'that', 13: 'in', 14: 'he', 15: 'was', 16: 'it', 17: 'his', 18: 'you', 19: 'with', 20: 'for', 21: 'had', 22: 'as', 23: ';', 24: '’', 25: 'is', 26: 'not', 27: '?', 28: 'but', 29: 'at', 30: 'my', 31: 'be', 32: '!', 33: 'him', 34: 'have', 35: 'this', 36: 'which', 37: 'her', 38: 'me', 39: 'we', 40: 'on', 41: 'by', 42: 'from', 43: '--', 44: 'said', 45: 'she', 46: 'they', 47: 'all', 48: 'were', 49: 'so', 50: 'there', 51: 'who', 52: 'one', 53: 'upon', 54: 'then', 55: 'no', 56: 'what', 57: 's', 58: 'will', 59: 'or', 60: 'would', 61: 'an', 62: 'them', 63: 'are', 64: 'if', 65: 'their', 66: 'man', 67: 'when', 68: 'your', 69: 'been', 70: 'now', 71: 'do', 72: 'out', 73: 'more', 74: 'could', 75: 'us', 76: 'into', 77: 'up', 78: 'some', 79: 'like', 80: 'very', 81: 'our', 82: 'did', 83: 'd', 84: "'", 85: 'two', 86: 'about', 87: 'little', 88: 'well', 89: 'great', 90: "'s", 

In [ ]:
# for genre in GLOBAL_genres:
#   print("Genre:", genre, "\n----------")
#   print("Unigrams (count =", ugc_dict[genre], "types =", len(list(ugf_dict[genre].items())), "):", str(ugf_dict[genre])[:10000])
#   print("Bigrams  (count =", bgc_dict[genre], "types =", len(list(bgf_dict[genre].items())), "):", str(bgf_dict[genre])[:10000])
#   print("Trigrams (count =", tgc_dict[genre], "types =", len(list(tgf_dict[genre].items())), "):", str(tgf_dict[genre])[:10000], "\n")


# Making the transmission matrices

## Initialising matrices with (t1, t2) -> t3 probabilities

In [ ]:
# makes the A matrix for the HMM model 

def make_init_A_mat(arg_tgf_smk_dict):
  a_mats = {}
  for genre, tf_dict in arg_tgf_smk_dict.items():
    num_rows = bgt_dict[genre]
    num_cols = ugt_dict[genre]
    a_mats[genre] = np.zeros((num_rows, num_cols))
    print(genre)
    for (tg, tgp) in tf_dict.items():
      bg = (tg[0], tg[1])
      ug = tg[2]
      if bg in bg_ridxd[genre] and ug in ug_ridxd[genre]:
        i = bg_ridxd[genre][bg]
        j = ug_ridxd[genre][ug]
        a_mats[genre][i, j] = tgp
  return a_mats

A_mats = make_init_A_mat(tgf_dict) # unsmoothed matrix A
# print(A_mats0) 

adventure
biography
childrens_fiction
christmas
detective
fantasy
horror
philosophy
poetry
science_fiction


## Add-k smoothing and unknown handling

In [ ]:
# smoothes the matrix
import math
k_value = 0.01

def smk_mat_A(arg_mats, arg_k):
  for (genre, mat) in arg_mats.items():
    v = bgt_dict[genre] # no. of unique bigrams in the genre 
    mat[:, 0] *= 0 # multiply column 0 so that the model never generates unknown words from bigrams 
    bgf = np.sum(mat, axis = 1) # bg frequency not counting (tok, tok, <UNK>)s 
    for i in range(len(mat)):
      # bg = bg_idxd(i) # bigram corresponding to the row of the matrix 
      mat[i] += arg_k
      mat[i] /= (bgf[i] + arg_k * v)
    mat[:, 0] *= 0.5 # multiply column 0 so the <UNKS> are strictly less preferred
    arg_mats[genre] = np.log(mat)

smk_mat_A(A_mats, k_value)
print_dict(A_mats)


{'adventure': array([[-16.71356385, -16.02041667, -16.02041667, ...,  -9.80381057,
        -16.02041667, -11.40529615],
       [-12.58039102, -11.88724384, -11.88724384, ..., -11.88724384,
        -11.88724384, -11.88724384],
       [-16.05445273, -15.36130555,  -1.95255944, ..., -15.36130555,
        -10.74618503, -10.05800064],
       ...,
       [-12.58039102, -11.88724384, -11.88724384, ..., -11.88724384,
        -11.88724384, -11.88724384],
       [-12.58107855, -11.88793137, -11.88793137, ..., -11.88793137,
        -11.88793137, -11.88793137],
       [-12.58039102, -11.88724384, -11.88724384, ..., -11.88724384,
        -11.88724384, -11.88724384]]), 'biography': array([[-16.23475824, -15.54161106, -10.92649054, ..., -15.54161106,
        -15.54161106, -15.54161106],
       [-12.39256051, -11.69941333, -11.69941333, ..., -11.69941333,
        -11.69941333, -11.69941333],
       [-15.94752107, -15.25437389, -15.25437389, ...,  -8.701866  ,
        -10.63925337, -15.25437389],
    

In [ ]:
GLOBAL_UNKs_sorted = {}
GLOBAL_UNKs_list = {}
for genre in GLOBAL_genres:
  GLOBAL_UNKs_sorted[genre] = {unk: freq for unk, freq in sorted(GLOBAL_UNKs[genre].items(), key=lambda item: item[1], reverse = True)}
  GLOBAL_UNKs_list[genre] = [unk for unk, freq in sorted(GLOBAL_UNKs[genre].items(), key=lambda item: item[1], reverse = True)]

print_ddict(GLOBAL_UNKs_sorted)
print_ddict(GLOBAL_UNKs_list)


('adventure', {'hunt': 127, 'week': 127, 'buried': 127, 'built': 127, 'slipped': 127, 'natives': 127, 'marût': 127, 'slight': 126, 'promised': 126, 'presented': 126, 'sorry': 126, 'cruel': 126, 'shame': 126, 'afternoon': 126, 'folk': 126, 'zulu': 126, 'drop': 125, 'sufficient': 125, 'astonishment': 125, 'complete': 125, 'sum': 125, 'telling': 125, 'pressed': 125, 'unfortunate': 125, 'affairs': 125, 'breakfast': 125, 'ate': 125, 'kitty': 125, 'storm': 125, 'former': 125, 'slain': 125, 'african': 125, 'blacks': 125, 'julian': 125, 'victoria': 125, 'ali': 125, 'cloak': 124, 'pray': 124, 'ear': 123, 'frightened': 123, 'closely': 123, 'alas': 123, 'hell': 123, 'accept': 123, 'report': 123, 'ladies': 123, 'mount': 123, 'finger': 123, 'ring': 123, 'gun': 123, 'zikali': 123, 'details': 122, 'meaning': 122, 'leading': 122, 'plainly': 122, 'generally': 122, 'clearly': 122, 'leaning': 122, 'horror': 122, 'animal': 122, 'spirits': 122, 'ocean': 122, 'fly': 121, 'situation': 121, 'calling': 121, '

In [ ]:
def sort_genres_mat(arg_A_mats):
  for (genre, mat) in arg_A_mats.items():
    arg_A_mats[genre] = np.argsort(-1 * mat, axis = 1) 

sort_genres_mat(A_mats)


In [ ]:
def get_random_UNK(arg_genre):
  randidx = int((randint(0, len(GLOBAL_UNKs_list[arg_genre]) - 1))**(0.5)) - 1
  return GLOBAL_UNKs_list[arg_genre][randidx]

# Generative Model: N-grams

In [ ]:
def check_end(sent):
  for sent_end in GLOBAL_sent_ends:
    if sent_end in sent:
      return False 
  return True
    

## Model 1: initial


---


Model 1: each token $w_t$ is generated by taking $argmax_w [{P(w_t|(w_{t-2}, w_{t-1}))}]$ i.e. selecting a token that has the maximum possibility of being generated given the prior two tokens. 


In [ ]:
from random import randint
from string import punctuation

# given bigram (n1, n2), returns the most likely n3 based on the sorted A matrix for the genre 
def get_ug_from_bg(arg_bg, arg_genre):
  bg = arg_bg
  if bg not in bg_ridxd[arg_genre]:
    bg = ("<UNK>", "<UNK>")
  bg_index = bg_ridxd[arg_genre][bg]
  ug_index = A_mats[arg_genre][bg_index][0]
  ug = ug_idxd[arg_genre][ug_index]
  # print(bg, ug)
  return ug

get_ug_from_bg(("he", "said"), "adventure")

punct_str = punctuation + "’"

def NB_make_sent_1(arg_keywords, arg_genre, arg_wordcount = 20, arg_sentcount = 3, arg_demo = False):
  if arg_demo == False:
    print("Generating", arg_sentcount, "sentence(s) of", arg_wordcount, "words, for genre:", arg_genre, "; starting with keyword(s):", str(arg_keywords), "\n----------")
  res = ""
  keywords = []
  for kw in arg_keywords:
    keywords.append((kw.lower()).strip())
  for s in range (0, arg_sentcount):
    sent_str = ""
    sent = ["<s>", "<s>"]
    start = 1
    if keywords[s] in ug_ridxd[arg_genre]:
      sent.append(keywords[s])
      start = 2
    elif keywords[s] != "" or keywords[s] != " ":
      sent.append("<UNK>")
      start = 2
    for i in range(start, arg_wordcount + 2):
      bg = (sent[i - 1], sent[i])
      ug = get_ug_from_bg(bg, arg_genre)
      sent.append(ug)
    # print(sent)
    no_unks = [tok if tok != "<UNK>" else keywords[s] for tok in sent]
    for t in range(0, len(no_unks)):
      tok = no_unks[t]
      if tok != "<s>" and tok != "</s>":
        if no_unks[t] not in punct_str:
          sent_str = sent_str + " " + tok
        else:
          if t < len(no_unks) - 1:
            sent_str = sent_str + tok
      
    res = res + (sent_str.strip()).capitalize() + "." + "\n"
  return res
  
for genre in GLOBAL_genres:
  print(NB_make_sent_1(["He", "she", "thEy"], genre, 25, 3))
  
    

Generating 3 sentence(s) of 25 words, for genre: adventure ; starting with keyword(s): ['He', 'she', 'thEy'] 
----------
He was a little, and the other, and the other, and the other, and the other, and the other, and.
She was a little, and the other, and the other, and the other, and the other, and the other, and.
They were not the least in the world, and the other, and the other, and the other, and the other, and.

Generating 3 sentence(s) of 25 words, for genre: biography ; starting with keyword(s): ['He', 'she', 'thEy'] 
----------
He was a man of the most important of the most important of the most important of the most important of the most important of the.
She was a man of the most important of the most important of the most important of the most important of the most important of the.
They were not to be a great deal of time, and the other hand, and the other hand, and the other hand.

Generating 3 sentence(s) of 25 words, for genre: childrens_fiction ; starting with keyword(s)

## Model 2: Gets rid of repeats 
Model 2 gets rid of repeats by selecting the next best unigram if the bigram was already seen. Model 2 selects the $n^{th}$ best token generated by the prior bigram, where $n$ is the number of times this bigram has been seen so far in the generated sentence.  

In [ ]:
# given bigram (n1, n2), returns the nth most likely n3 based on the sorted A matrix for the genre 
def get_nth_ug(arg_bg, arg_genre, arg_n):
  bg = arg_bg
  if bg not in bg_ridxd[arg_genre]:
    bg = ("<UNK>", "<UNK>")
  bg_index = bg_ridxd[arg_genre][bg]
  # ug_index = A_sorted_mats[arg_genre][bg_index][(arg_n - 1)]
  ug_index = A_mats[arg_genre][bg_index][(arg_n - 1) % ugt_dict[arg_genre]]
  ug = ug_idxd[arg_genre][ug_index]
  # print(bg, ug)
  return ug

def NB_make_sent_2(arg_keywords, arg_genre, arg_wordcount = 20, arg_sentcount = 3, arg_demo = False):
  if arg_demo == False:
    print("Generating", arg_sentcount, "sentence(s) of", arg_wordcount, "words, for genre:", arg_genre, "; starting with keyword(s):", str(arg_keywords), "\n----------")
  res = ""
  keywords = []
  # change from model 1
  used_bg = {}
  for kw in arg_keywords:
    keywords.append((kw.lower()).strip())
  for s in range (0, arg_sentcount):
    sent_str = ""
    sent = ["<s>", "<s>"]
    start = 1
    if keywords[s] in ug_ridxd[arg_genre]:
      sent.append(keywords[s])
      start = 2
    elif keywords[s] != "" or keywords[s] != " ":
      sent.append("<UNK>")
      start = 2
    for i in range(start, arg_wordcount + 2):
      if check_end(sent):
        bg = (sent[i - 1], sent[i])
        add_val_to_dict(used_bg, bg, 1)
        ug = get_nth_ug(bg, arg_genre, used_bg[bg])
        sent.append(ug)
    # if "." not in sent:
    #   sent.append(".")
    # print(sent)
    no_unks = [tok if tok != "<UNK>" else get_random_UNK(genre) for tok in sent]
    for t in range(0, len(no_unks)):
      tok = no_unks[t]
      if tok != "<s>" and tok != "</s>":
        if no_unks[t] not in punct_str:
          sent_str = sent_str + " " + tok
        else:
          if t < len(no_unks) - 1:
            sent_str = sent_str + tok
    if sent_str[len(sent_str) - 1] not in punct_str:
      sent_str += "."
    res = res + (sent_str.strip()).capitalize() + "\n"
  return res

for genre in GLOBAL_genres:
  print(NB_make_sent_2(["He", "She", "They"], genre, 25, 3), "\n")

Generating 3 sentence(s) of 25 words, for genre: adventure ; starting with keyword(s): ['He', 'She', 'They'] 
----------
He was a little, and the other, and i am not a man of the great apes.
She was a man who had been a great deal of the king," said the doctor," i am a good deal of.
They were not the least in the world, and that is to say," he said," replied the doctor.
 

Generating 3 sentence(s) of 25 words, for genre: biography ; starting with keyword(s): ['He', 'She', 'They'] 
----------
He was a man of the most important of the people, and the other hand, and i have been a very different from the.
She was a great deal of time, and that the king, and he was not a little later, the most powerful of all.
They were not to be a great number of the king’ s, and to the king of the united states, and, in.
 

Generating 3 sentence(s) of 25 words, for genre: childrens_fiction ; starting with keyword(s): ['He', 'She', 'They'] 
----------
He was a very good," said jack," he said," the sheik s

## Model 3

In [ ]:
def get_nth_ug_3(arg_bg, arg_genre, arg_n):
  bg = arg_bg
  if bg not in bg_ridxd[arg_genre]:
    bg = ("<UNK>", "<UNK>")
  bg_index = bg_ridxd[arg_genre][bg]
  ug_index = A_mats[arg_genre][bg_index][(arg_n)]
  ug = ug_idxd[arg_genre][ug_index]
  return ug

def NB_make_sent_3(arg_keywords, arg_genre, arg_wordcount = 20, arg_sentcount = 3, arg_top_n = 2, arg_demo = False):
  if arg_demo == False:
    print("Generating", arg_sentcount, "sentence(s) of", arg_wordcount, "words, for genre:", arg_genre, "; starting with keyword(s):", str(arg_keywords), "\n----------")
  res = ""
  keywords = []
  # change from model 1
  used_bg = {}
  used_bg_idx = {}
  for kw in arg_keywords:
    keywords.append((kw.lower()).strip())
  for s in range (0, arg_sentcount):
    sent_str = ""
    sent = ["<s>", "<s>"]
    start = 1
    if keywords[s] in ug_ridxd[arg_genre]:
      sent.append(keywords[s])
      start = 2
    elif keywords[s] != "" or keywords[s] != " ":
      sent.append("<UNK>")
      start = 2
    for i in range(start, arg_wordcount + 2):
      if check_end(sent):
        bg = (sent[i - 1], sent[i])
        add_val_to_dict(used_bg, bg, 1)
        init_dict_key(used_bg_idx, bg, [])
        offset = int(used_bg[bg]/arg_top_n) * arg_top_n
        idxs = [x for x in range(offset, offset + arg_top_n) if x not in used_bg_idx[bg]]
        # print(idxs)
        idx = idxs[randint(0, len(idxs) -1)]
        # print(len(idxs), bg, idx, idxs, offset, used_bg, used_bg_idx)
        ug = get_nth_ug_3(bg, arg_genre, idx) # because get_nth_ug gets the (n-1)th in the definition code
        used_bg_idx[bg].append(idx)
        sent.append(ug)
    no_unks = [tok if tok != "<UNK>" else get_random_UNK(genre) for tok in sent]
    for t in range(0, len(no_unks)):
      tok = no_unks[t]
      if tok != "<s>" and tok != "</s>":
        if no_unks[t] not in punct_str:
          sent_str = sent_str + " " + tok
        else:
          if t < len(no_unks) - 1:
            sent_str = sent_str + tok
    if sent_str[len(sent_str) - 1] not in punct_str:
      sent_str += "."
    res = res + (sent_str.strip()).capitalize() + "\n"
  return res

for genre in GLOBAL_genres:
  print(NB_make_sent_3(["He", "She", "They"], genre, 25, 3, 2), "\n")

Generating 3 sentence(s) of 25 words, for genre: adventure ; starting with keyword(s): ['He', 'She', 'They'] 
----------
He was not the less, and i will tell him that he was, as i have been the cause.
She had not the man who has the courage of the king," said he," replied d’ ye see," he.
They are not the first time i was a little," and i was to go to the ground, the doctor," she.
 

Generating 3 sentence(s) of 25 words, for genre: biography ; starting with keyword(s): ['He', 'She', 'They'] 
----------
He was a man who had been the most powerful of these, however, was the first time, and the duc d’ a.
She was not a little later than the poor, dear," said he, as the first to the people, who was a.
They had no doubt, but the fact that the king’ s letter to the house of the most part as less likely to have.
 

Generating 3 sentence(s) of 25 words, for genre: childrens_fiction ; starting with keyword(s): ['He', 'She', 'They'] 
----------
He had not yet returned, with a smile, while the rest

### DEMO

In [ ]:
print("genre = \"random\" #@param", str(["random"] + GLOBAL_genres))
print(most_freq)

genre = "random" #@param ['random', 'adventure', 'biography', 'childrens_fiction', 'christmas', 'detective', 'fantasy', 'horror', 'philosophy', 'poetry', 'science_fiction']
{'adventure': {',': 158869, 'the': 127010, '.': 83044, 'of': 62428, 'and': 62319, 'to': 51783, 'a': 44117, 'i': 36462, 'that': 31382, 'in': 30977, 'he': 26791, 'was': 24239, 'it': 22893, '“': 22080, 'his': 21931, '”': 21451, 'you': 17270, 'with': 16507, 'for': 15870, 'had': 15519, 'as': 15488, ';': 15275, '’': 14358, 'is': 14135, 'not': 14001, '?': 13795, 'but': 13276, 'at': 13112, "''": 12132, '``': 11151, 'my': 11094, 'be': 10690, '!': 10680, 'him': 10498, 'have': 10339, 'this': 10071, 'which': 9805, 'her': 9723, 'me': 9544, 'we': 9530, 'on': 9530, 'by': 9201, 'from': 9028, '--': 8844, 'said': 8740, 'she': 8408, 'they': 8232, 'all': 7937, 'were': 7650, 'so': 7565, 'there': 6758, 'who': 6530, 'one': 6472, 'upon': 6260, 'then': 6072, 'no': 6010, 'what': 5971, 's': 5944, 'will': 5824, 'or': 5780, 'would': 5698, 'an':

In [ ]:
#@title Story Generator
#@markdown Generates sentence(s) starting with keyword(s) based on a user-selected genre. 

keywords = "The, The, Thus" #@param {type:"string"}
# replace the line below with the output above
genre = "all" #@param ['random', 'all', 'adventure', 'biography', 'childrens_fiction', 'christmas', 'detective', 'fantasy', 'horror', 'philosophy', 'poetry', 'science_fiction']
num_sents =  3#@param {type:"integer"}
max_sent_len =  200#@param {type:"integer"}
force_max_sent_len = "Off" 
model = "NB_Model_2" #@param ["NB_Model_1", "NB_Model_2", "NB_Model_3"]

from IPython.display import HTML

kw = [w.strip() for w in (keywords.split(","))]
if len(kw) != num_sents:
  if len(kw) > num_sents:
    kw = kw[ : num_sents]
  else:
    diff = num_sents - len(kw)
    for d in range(diff):
      kw.append(" ")
      
chosen_genres = [genre]
if genre == "random":
  chosen_genres = [GLOBAL_genres[randint(0, len(GLOBAL_genres) - 1)]]
elif genre == "all":
  chosen_genres = [g for g in GLOBAL_genres]

# Generate nice display
metadata = ""
metadata += '<div style="font-family:courier new; font-size:100%;">'
metadata += "<p> Model: " + model
# metadata += "; Likelihood = " + str(likelihood) +"</p>"

display(HTML(metadata))
for genre in chosen_genres:
  input = ""
  input += '<div style="font-family:verdana; font-size:100%;">'
  input += "<p> Genre: <strong>" + genre + "</strong>; Keyword(s): <strong>" + str(kw) + "</strong>; Sentences: <strong>" + str(num_sents) + "</strong>; Max sentence length: <strong>" + str(max_sent_len) + "</strong></p>"
  if len(chosen_genres) == 1: input = "<h3>User Input Parameters:</h3>" + input

  likelihood = 0

  model_dict = {"NB_Model_1" : NB_make_sent_1(kw, genre, max_sent_len, num_sents, True), 
                "NB_Model_2" : NB_make_sent_2(kw, genre, max_sent_len, num_sents, True), 
                "NB_Model_3" : NB_make_sent_3(kw, genre, max_sent_len, num_sents, 2, True)
                }
  sents = (model_dict[model]).split("\n")[ : -1]

  display(HTML(input))
  if len(chosen_genres) == 1: print("––––––––––")
  for sent in sents:
    output = ""
    output += "<p>" + sent + "</p>"
    display(HTML(output))
  print()

